# Database Query

This code is supposed to download the data from the DESI database daily and store relevant information in a fits file. For now I just query the data once and then put it in a fits file. From there, one could load the data, download all the data from the database and only add whats new on a daily basis.

In [3]:
from astropy.io import fits
from astropy.table import Table, hstack, vstack
from astropy import table

import numpy as np

import gasel as gs

from tqdm import tqdm

In [2]:
# connect to db
cursor = gs.connect_to_time_db()

In [4]:
diameter = 4

In [3]:
full_data = []

for x in tqdm(np.arange(0, 370, diameter)): # could improve this since 0 = 360... (was 370)
    #print("x: ", x)
    for y in np.arange(-20, 90, diameter): # (was 100)
        #print("x & y", x,y)
        #print(y)
        rows = gs.get_data_rad_search(x,y, diameter/2, cursor)
        #data = gs.make_list(rows)
        #print(rows[:2])
        if rows != None:
            #print(f"data {len(rows)}")
            #print(full_data)
            #print(data)
            #print(rows[:2])
            for obj in rows:
                
                full_data.append(obj)
        
full_table = Table(rows = full_data, names = gs.std_names)


full_table[:5]

100%|██████████| 93/93 [28:24<00:00, 18.32s/it]


TARGETID,TARGET_RA,TARGET_DEC,LASTNIGHT,Z,ZERR,ZWARN,FLUX_G,FLUX_R,FLUX_Z,SPECTYPE,BGS_TARGET,EBV,SERSIC
int64,float64,float64,int64,float64,float64,int64,float64,float64,float64,str6,int64,float64,float64
39627461389979471,0.199208396611172,-13.4284533588966,20211020,0.0842981615211128,1.02751859803205e-05,0,349.176239013672,151.578369140625,657.703186035156,GALAXY,131074,0.0286298077553511,1.75290405750275
39627461394171068,0.324188154725867,-13.3757445122443,20211020,0.358443849991424,6.26159169498368e-05,0,15.7543725967407,3.10386729240417,35.017635345459,GALAXY,131074,0.0287617538124323,4.0
39627461394173293,0.423061498719655,-13.3821980272798,20211020,7.96335573109056e-06,2.72051014406708e-06,0,58.3771476745605,25.3495140075684,91.9729995727539,STAR,0,0.0290689337998629,0.0
39627467278778997,0.54074701108748,-13.1672162663649,20211020,0.368523065341714,2.55883523529775e-05,0,15.623589515686,5.02074289321899,32.4087829589844,GALAXY,131074,0.0333173200488091,1.0
616088219582202155,0.626179874156068,-13.3202664780153,20211020,0.187326155709928,7.97282091423762e-06,1,0.0,0.0,0.0,GALAXY,0,0.0,0.0


In [4]:
full_table = table.unique(full_table, keys = "TARGETID")

In [24]:
len(full_data)

56038946

In [10]:
full_table.write('/global/homes/j/jgassert/full_table.ecsv', overwrite=True)  

In [11]:
print("Stuff")

Stuff


In [5]:
full_data2 = []

for x in tqdm(np.arange(2, 368, diameter)): # (was 360)
    #print("x: ", x)
    for y in np.arange(-18, 86, diameter): # (was 90)
        #print("x & y", x,y)
        #print(y)
        rows2 = gs.get_data_rad_search(x,y, diameter/2, cursor)
        #data = gs.make_list(rows)
        #print(rows[:2])
        if rows2 != None:
            #print(f"data {len(rows)}")
            #print(full_data)
            #print(data)
            #print(rows[:2])
            for obj2 in rows2:
                
                full_data2.append(obj2)

100%|██████████| 92/92 [26:48<00:00, 17.48s/it]


In [6]:
full_table2 = Table(rows = full_data2, names = gs.std_names)

In [7]:
full_table2 = table.unique(full_table2, keys = "TARGETID")

In [8]:
len(full_table2)

36559680

In [9]:
full_table2.write('/global/homes/j/jgassert/full_table2.ecsv', overwrite=True)  

In [22]:
full = 0

In [ ]:
full = hstack([full_table, full_table2])

In [26]:
len(full)

36638229

In [21]:
full[:6]

TARGETID_1,TARGET_RA_1,TARGET_DEC_1,LASTNIGHT_1,Z_1,ZERR_1,ZWARN_1,FLUX_G_1,FLUX_R_1,FLUX_Z_1,SPECTYPE_1,BGS_TARGET_1,EBV_1,SERSIC_1,TARGETID_2,TARGET_RA_2,TARGET_DEC_2,LASTNIGHT_2,Z_2,ZERR_2,ZWARN_2,FLUX_G_2,FLUX_R_2,FLUX_Z_2,SPECTYPE_2,BGS_TARGET_2,EBV_2,SERSIC_2
int64,float64,float64,int64,float64,float64,int64,float64,float64,float64,str6,int64,float64,float64,int64,float64,float64,int64,float64,float64,int64,float64,float64,float64,str6,int64,float64,float64
-833389526,227.433272279891,66.6519046819295,20230427,1.1824834644376,9.79136990918098e-05,5,0.0,0.0,0.0,GALAXY,0,0.0290863439440727,0.0,-833389526,227.433272279891,66.6519046819295,20230427,1.1824834644376,9.79136990918098e-05,5,0.0,0.0,0.0,GALAXY,0,0.0290863439440727,0.0
-833389518,228.204498084663,66.7769162408949,20230427,1.01705132530733,0.000198853226529452,5,0.0,0.0,0.0,GALAXY,0,0.0315619334578514,0.0,-833389518,228.204498084663,66.7769162408949,20230427,1.01705132530733,0.000198853226529452,5,0.0,0.0,0.0,GALAXY,0,0.0315619334578514,0.0
-833389510,227.642453626374,66.7226250371644,20230427,1.20045807712059,0.000130270720806434,5,0.0,0.0,0.0,GALAXY,0,0.0310118906199932,0.0,-833389510,227.642453626374,66.7226250371644,20230427,1.20045807712059,0.000130270720806434,5,0.0,0.0,0.0,GALAXY,0,0.0310118906199932,0.0
-833389492,227.755926256736,66.7768026589679,20230427,1.39280058555401,0.000191373523674881,2053,0.0,0.0,0.0,GALAXY,0,0.0316010527312756,0.0,-833389492,227.755926256736,66.7768026589679,20230427,1.39280058555401,0.000191373523674881,2053,0.0,0.0,0.0,GALAXY,0,0.0316010527312756,0.0
-833389491,227.851676937586,66.7933443247276,20230427,0.747688478964211,8.9141824351558e-05,5,0.0,0.0,0.0,GALAXY,0,0.0321089513599873,0.0,-833389491,227.851676937586,66.7933443247276,20230427,0.747688478964211,8.9141824351558e-05,5,0.0,0.0,0.0,GALAXY,0,0.0321089513599873,0.0
-833389472,227.867586316838,66.8320572673735,20230427,1.19571717482174,0.000116999729896098,2053,0.0,0.0,0.0,GALAXY,0,0.0323736257851124,0.0,-833389472,227.867586316838,66.8320572673735,20230427,1.19571717482174,0.000116999729896098,2053,0.0,0.0,0.0,GALAXY,0,0.0323736257851124,0.0


In [20]:
full = table.unique(full, keys = "TARGETID")

KeyError: 'TARGETID'

In [ ]:
len(full)

In [2]:
from astropy.io import ascii


In [2]:
data1 = ascii.read("/global/homes/j/jgassert/full_table.ecsv") 

In [3]:
data2 = ascii.read("/global/homes/j/jgassert/full_table2.ecsv") 

In [12]:
data1[:5]

TARGETID,TARGET_RA,TARGET_DEC,LASTNIGHT,Z,ZERR,ZWARN,FLUX_G,FLUX_R,FLUX_Z,SPECTYPE,BGS_TARGET,EBV,SERSIC
int64,float64,float64,int64,float64,float64,int64,float64,float64,float64,str6,int64,float64,float64
-833389526,227.433272279891,66.6519046819295,20230427,1.1824834644376,9.79136990918098e-05,5,0.0,0.0,0.0,GALAXY,0,0.0290863439440727,0.0
-833389518,228.204498084663,66.7769162408949,20230427,1.01705132530733,0.000198853226529452,5,0.0,0.0,0.0,GALAXY,0,0.0315619334578514,0.0
-833389510,227.642453626374,66.7226250371644,20230427,1.20045807712059,0.000130270720806434,5,0.0,0.0,0.0,GALAXY,0,0.0310118906199932,0.0
-833389492,227.755926256736,66.7768026589679,20230427,1.39280058555401,0.000191373523674881,2053,0.0,0.0,0.0,GALAXY,0,0.0316010527312756,0.0
-833389491,227.851676937586,66.7933443247276,20230427,0.747688478964211,8.9141824351558e-05,5,0.0,0.0,0.0,GALAXY,0,0.0321089513599873,0.0


In [9]:
data = vstack([data1, data2])

In [10]:
data[:5]

TARGETID,TARGET_RA,TARGET_DEC,LASTNIGHT,Z,ZERR,ZWARN,FLUX_G,FLUX_R,FLUX_Z,SPECTYPE,BGS_TARGET,EBV,SERSIC
int64,float64,float64,int64,float64,float64,int64,float64,float64,float64,str6,int64,float64,float64
-833389526,227.433272279891,66.6519046819295,20230427,1.1824834644376,9.79136990918098e-05,5,0.0,0.0,0.0,GALAXY,0,0.0290863439440727,0.0
-833389518,228.204498084663,66.7769162408949,20230427,1.01705132530733,0.000198853226529452,5,0.0,0.0,0.0,GALAXY,0,0.0315619334578514,0.0
-833389510,227.642453626374,66.7226250371644,20230427,1.20045807712059,0.000130270720806434,5,0.0,0.0,0.0,GALAXY,0,0.0310118906199932,0.0
-833389492,227.755926256736,66.7768026589679,20230427,1.39280058555401,0.000191373523674881,2053,0.0,0.0,0.0,GALAXY,0,0.0316010527312756,0.0
-833389491,227.851676937586,66.7933443247276,20230427,0.747688478964211,8.9141824351558e-05,5,0.0,0.0,0.0,GALAXY,0,0.0321089513599873,0.0


In [11]:
data = table.unique(data, keys = "TARGETID")

In [12]:
len(data1), len(data2), len(data)

(36638229, 36559680, 43953830)

In [13]:
data.write('/global/homes/j/jgassert/data.ecsv', overwrite=False)  

In [4]:
data = ascii.read("/global/homes/j/jgassert/data.ecsv") 

In [5]:
len(data)

43953830

In [6]:
data[:5]

TARGETID,TARGET_RA,TARGET_DEC,LASTNIGHT,Z,ZERR,ZWARN,FLUX_G,FLUX_R,FLUX_Z,SPECTYPE,BGS_TARGET,EBV,SERSIC
int64,float64,float64,int64,float64,float64,int64,float64,float64,float64,str6,int64,float64,float64
-833389526,227.433272279891,66.6519046819295,20230427,1.1824834644376,9.79136990918098e-05,5,0.0,0.0,0.0,GALAXY,0,0.0290863439440727,0.0
-833389518,228.204498084663,66.7769162408949,20230427,1.01705132530733,0.000198853226529452,5,0.0,0.0,0.0,GALAXY,0,0.0315619334578514,0.0
-833389510,227.642453626374,66.7226250371644,20230427,1.20045807712059,0.000130270720806434,5,0.0,0.0,0.0,GALAXY,0,0.0310118906199932,0.0
-833389492,227.755926256736,66.7768026589679,20230427,1.39280058555401,0.000191373523674881,2053,0.0,0.0,0.0,GALAXY,0,0.0316010527312756,0.0
-833389491,227.851676937586,66.7933443247276,20230427,0.747688478964211,8.9141824351558e-05,5,0.0,0.0,0.0,GALAXY,0,0.0321089513599873,0.0
